In [1]:
from pynq import Overlay, allocate

In [2]:
import numpy as np

In [3]:
import time

In [4]:
ol = Overlay("./design_1.bit")

In [5]:
ol.ip_dict.keys()

dict_keys(['cnn_0', 'axi_dma_0', 'axi_dma_1', 'processing_system7_0'])

In [6]:
dma0 = ol.axi_dma_0
dma1 = ol.axi_dma_1
hls_ip = ol.cnn_0

In [7]:
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [8]:
img_h = 30
ing_w = 30
img_d = 3
image_size = 2700

In [9]:
image_data = [
    [[[236, 240, 255]], [[230, 231, 255]], [[119, 125, 169]], [[71, 77, 95]], [[42, 45, 50]], [[28, 29, 28]], [[23, 23, 26]], [[24, 23, 26]], [[23, 22, 25]], [[24, 21, 25]], [[24, 21, 25]], [[26, 22, 26]], [[31, 26, 27]], [[31, 27, 28]], [[32, 27, 29]], [[30, 26, 28]], [[31, 26, 28]], [[29, 25, 27]], [[29, 24, 25]], [[27, 24, 24]], [[26, 25, 25]], [[26, 25, 26]], [[29, 28, 29]], [[32, 30, 34]], [[36, 31, 35]], [[43, 36, 37]], [[48, 41, 41]], [[52, 46, 46]], [[64, 52, 54]], [[108, 79, 49]]],
    [[[235, 238, 255]], [[233, 235, 255]], [[119, 126, 165]], [[63, 70, 89]], [[27, 32, 41]], [[25, 24, 27]], [[25, 22, 26]], [[23, 21, 24]], [[23, 21, 24]], [[24, 21, 25]], [[24, 21, 25]], [[25, 21, 25]], [[31, 25, 27]], [[36, 29, 29]], [[37, 30, 30]], [[36, 30, 29]], [[36, 29, 28]], [[35, 29, 29]], [[34, 29, 29]], [[32, 29, 29]], [[28, 27, 27]], [[28, 26, 27]], [[30, 28, 30]], [[33, 31, 34]], [[36, 30, 34]], [[43, 35, 37]], [[48, 40, 40]], [[51, 46, 47]], [[62, 52, 56]], [[104, 80, 51]]],
    [[[242, 241, 255]], [[237, 239, 255]], [[123, 132, 164]], [[57, 62, 80]], [[28, 28, 37]], [[26, 24, 28]], [[25, 23, 26]], [[24, 21, 23]], [[24, 21, 23]], [[25, 22, 24]], [[23, 21, 24]], [[25, 22, 25]], [[30, 25, 27]], [[38, 32, 33]], [[43, 36, 39]], [[43, 35, 38]], [[42, 35, 38]], [[41, 36, 39]], [[41, 37, 41]], [[40, 37, 42]], [[34, 31, 33]], [[30, 27, 29]], [[34, 32, 35]], [[33, 31, 34]], [[35, 30, 32]], [[42, 35, 37]], [[47, 39, 41]], [[52, 45, 46]], [[66, 52, 56]], [[106, 78, 63]]],
    [[[230, 232, 254]], [[243, 243, 255]], [[138, 138, 163]], [[71, 72, 87]], [[28, 28, 36]], [[25, 25, 28]], [[24, 23, 25]], [[24, 22, 24]], [[23, 21, 23]], [[23, 20, 23]], [[24, 21, 24]], [[24, 21, 24]], [[27, 23, 25]], [[34, 32, 35]], [[39, 36, 44]], [[40, 36, 44]], [[39, 36, 43]], [[38, 35, 41]], [[37, 35, 42]], [[34, 33, 40]], [[32, 30, 34]], [[29, 27, 30]], [[30, 28, 33]], [[31, 30, 33]], [[34, 30, 32]], [[40, 34, 36]], [[47, 40, 41]], [[49, 44, 46]], [[61, 50, 54]], [[100, 75, 69]]],
    [[[228, 233, 254]], [[247, 247, 255]], [[142, 144, 161]], [[77, 81, 94]], [[28, 32, 38]], [[27, 28, 29]], [[24, 24, 26]], [[23, 23, 25]], [[23, 21, 24]], [[24, 21, 24]], [[25, 21, 25]], [[25, 21, 25]], [[26, 23, 24]], [[25, 24, 26]], [[27, 25, 28]], [[28, 25, 28]], [[27, 25, 27]], [[26, 24, 26]], [[24, 23, 25]], [[26, 24, 27]], [[26, 24, 28]], [[26, 24, 27]], [[28, 26, 29]], [[32, 29, 33]], [[34, 29, 32]], [[40, 34, 34]], [[46, 39, 39]], [[48, 43, 44]], [[58, 49, 52]], [[87, 68, 56]]],
    [[[227, 232, 252]], [[247, 251, 255]], [[130, 148, 160]], [[72, 87, 102]], [[54, 63, 74]], [[38, 43, 52]], [[31, 34, 42]], [[28, 30, 47]], [[24, 28, 52]], [[22, 25, 35]], [[25, 22, 27]], [[25, 21, 25]], [[27, 23, 26]], [[27, 24, 26]], [[27, 24, 26]], [[27, 23, 25]], [[27, 23, 25]], [[27, 23, 26]], [[26, 23, 25]], [[26, 24, 25]], [[25, 23, 25]], [[26, 23, 27]], [[31, 27, 30]], [[32, 28, 32]], [[34, 29, 32]], [[40, 34, 34]], [[45, 38, 38]], [[50, 44, 45]], [[58, 50, 54]], [[78, 67, 64]]],
    [[[234, 237, 255]], [[240, 254, 255]], [[119, 179, 182]], [[48, 88, 89]], [[78, 82, 89]], [[68, 74, 87]], [[60, 68, 87]], [[100, 108, 148]], [[112, 151, 173]], [[22, 63, 87]], [[20, 24, 35]], [[25, 22, 30]], [[28, 25, 30]], [[28, 26, 29]], [[27, 25, 27]], [[27, 24, 26]], [[26, 23, 26]], [[26, 24, 27]], [[26, 23, 25]], [[25, 22, 24]], [[26, 23, 25]], [[27, 24, 26]], [[28, 25, 29]], [[31, 28, 32]], [[34, 29, 32]], [[40, 33, 34]], [[46, 38, 39]], [[52, 45, 47]], [[58, 51, 55]], [[72, 63, 64]]],
    [[[232, 235, 255]], [[247, 255, 255]], [[125, 160, 149]], [[142, 165, 133]], [[115, 116, 106]], [[167, 170, 183]], [[200, 204, 240]], [[248, 255, 255]], [[99, 129, 219]], [[12, 42, 86]], [[25, 28, 51]], [[30, 28, 46]], [[31, 28, 38]], [[29, 28, 34]], [[27, 25, 31]], [[29, 25, 29]], [[27, 24, 26]], [[26, 24, 27]], [[26, 24, 27]], [[24, 22, 25]], [[26, 23, 25]], [[27, 25, 27]], [[26, 24, 27]], [[27, 25, 27]], [[31, 27, 29]], [[38, 32, 35]], [[46, 39, 42]], [[49, 42, 46]], [[57, 49, 52]], [[65, 55, 48]]],
    [[[227, 228, 248]], [[255, 255, 255]], [[255, 255, 252]], [[255, 255, 245]], [[255, 255, 251]], [[255, 255, 255]], [[231, 231, 255]], [[124, 131, 188]], [[42, 50, 94]], [[26, 32, 64]], [[29, 27, 51]], [[35, 33, 51]], [[36, 34, 44]], [[31, 30, 36]], [[31, 28, 34]], [[31, 26, 31]], [[27, 24, 28]], [[25, 24, 27]], [[26, 24, 27]], [[26, 24, 26]], [[29, 26, 28]], [[28, 26, 28]], [[24, 23, 26]], [[27, 25, 28]], [[32, 29, 31]], [[40, 35, 36]], [[47, 40, 42]], [[48, 42, 45]], [[54, 46, 51]], [[62, 50, 44]]],
    [[[198, 182, 201]], [[239, 233, 241]], [[255, 239, 246]], [[232, 209, 219]], [[194, 172, 185]], [[164, 139, 159]], [[114, 102, 150]], [[44, 58, 97]], [[27, 31, 55]], [[29, 28, 52]], [[33, 33, 54]], [[42, 41, 59]], [[39, 38, 46]], [[36, 35, 39]], [[38, 34, 39]], [[41, 32, 38]], [[34, 30, 35]], [[27, 25, 29]], [[26, 25, 28]], [[28, 26, 29]], [[32, 28, 30]], [[30, 28, 30]], [[27, 24, 27]], [[29, 26, 29]], [[34, 30, 32]], [[41, 36, 37]], [[46, 39, 40]], [[46, 41, 44]], [[47, 42, 47]], [[55, 48, 44]]],
    [[[76, 71, 87]], [[77, 75, 99]], [[88, 80, 107]], [[77, 75, 99]], [[65, 69, 98]], [[77, 72, 102]], [[59, 55, 74]], [[24, 29, 49]], [[27, 28, 51]], [[36, 34, 52]], [[54, 50, 66]], [[50, 51, 66]], [[39, 43, 49]], [[42, 42, 45]], [[45, 43, 48]], [[43, 39, 45]], [[34, 33, 39]], [[29, 29, 33]], [[30, 30, 33]], [[31, 30, 33]], [[31, 30, 32]], [[29, 28, 32]], [[25, 23, 28]], [[28, 25, 27]], [[34, 29, 31]], [[40, 34, 36]], [[45, 38, 40]], [[46, 40, 42]], [[46, 41, 45]], [[53, 48, 46]]],
    [[[80, 84, 100]], [[76, 84, 113]], [[93, 95, 124]], [[75, 87, 106]], [[49, 73, 91]], [[48, 59, 78]], [[36, 39, 58]], [[29, 27, 50]], [[30, 28, 51]], [[44, 41, 57]], [[68, 62, 74]], [[61, 61, 69]], [[57, 61, 68]], [[60, 59, 65]], [[62, 61, 64]], [[61, 61, 66]], [[61, 61, 67]], [[58, 58, 63]], [[55, 55, 58]], [[54, 54, 57]], [[50, 49, 59]], [[32, 33, 44]], [[24, 23, 29]], [[26, 24, 26]], [[33, 28, 29]], [[40, 34, 35]], [[44, 36, 38]], [[45, 40, 42]], [[45, 40, 42]], [[46, 42, 39]]],
    [[[79, 85, 100]], [[95, 103, 123]], [[90, 86, 106]], [[87, 94, 110]], [[49, 72, 88]], [[38, 49, 67]], [[28, 32, 54]], [[28, 27, 50]], [[37, 32, 54]], [[61, 56, 68]], [[68, 68, 76]], [[68, 67, 72]], [[72, 68, 74]], [[72, 68, 73]], [[70, 68, 71]], [[71, 70, 75]], [[74, 73, 78]], [[74, 74, 79]], [[72, 73, 77]], [[73, 71, 76]], [[71, 69, 78]], [[43, 48, 57]], [[24, 26, 32]], [[29, 28, 31]], [[34, 31, 33]], [[42, 37, 38]], [[43, 36, 37]], [[42, 39, 40]], [[40, 38, 41]], [[43, 40, 38]]],
    [[[48, 55, 69]], [[54, 61, 72]], [[58, 65, 77]], [[51, 69, 86]], [[52, 82, 101]], [[35, 46, 67]], [[28, 29, 52]], [[29, 28, 50]], [[44, 40, 58]], [[69, 64, 72]], [[70, 69, 74]], [[71, 69, 75]], [[72, 70, 74]], [[72, 71, 75]], [[72, 70, 73]], [[72, 70, 73]], [[72, 70, 75]], [[72, 70, 76]], [[72, 70, 76]], [[73, 71, 77]], [[73, 72, 77]], [[61, 62, 68]], [[52, 55, 62]], [[44, 49, 53]], [[46, 48, 51]], [[44, 43, 44]], [[44, 38, 39]], [[42, 39, 41]], [[41, 39, 42]], [[43, 40, 39]]],
    [[[49, 74, 83]], [[74, 96, 104]], [[98, 113, 130]], [[109, 124, 155]], [[113, 128, 152]], [[54, 59, 78]], [[30, 29, 52]], [[29, 28, 46]], [[54, 50, 63]], [[73, 69, 74]], [[72, 69, 73]], [[72, 69, 76]], [[73, 71, 76]], [[73, 71, 76]], [[72, 70, 74]], [[73, 69, 74]], [[72, 70, 75]], [[73, 71, 75]], [[73, 71, 75]], [[74, 72, 78]], [[74, 72, 77]], [[73, 72, 77]], [[71, 71, 79]], [[70, 73, 75]], [[62, 68, 67]], [[49, 52, 52]], [[48, 43, 44]], [[43, 40, 43]], [[41, 39, 44]], [[43, 41, 41]]],
    [[[101, 149, 182]], [[119, 161, 174]], [[157, 177, 189]], [[198, 202, 227]], [[128, 120, 143]], [[52, 48, 70]], [[30, 28, 54]], [[29, 28, 49]], [[52, 49, 63]], [[74, 70, 78]], [[74, 69, 74]], [[74, 71, 77]], [[74, 71, 76]], [[73, 71, 75]], [[73, 70, 75]], [[73, 69, 74]], [[72, 70, 76]], [[73, 72, 76]], [[73, 72, 76]], [[74, 72, 76]], [[74, 71, 76]], [[75, 72, 78]], [[75, 72, 79]], [[83, 82, 76]], [[103, 114, 110]], [[61, 69, 90]], [[48, 44, 56]], [[42, 41, 42]], [[41, 40, 42]], [[44, 42, 40]]],
    [[[116, 136, 172]], [[105, 121, 135]], [[88, 100, 107]], [[73, 77, 94]], [[66, 63, 83]], [[43, 42, 66]], [[28, 28, 56]], [[29, 28, 52]], [[51, 47, 64]], [[74, 68, 78]], [[75, 69, 75]], [[73, 70, 75]], [[73, 71, 77]], [[73, 71, 76]], [[72, 70, 76]], [[73, 70, 76]], [[72, 71, 76]], [[73, 72, 76]], [[73, 72, 77]], [[74, 72, 76]], [[74, 71, 77]], [[75, 72, 78]], [[78, 74, 81]], [[83, 84, 76]], [[117, 141, 140]], [[69, 87, 125]], [[48, 45, 66]], [[45, 42, 44]], [[44, 41, 44]], [[44, 41, 41]]],
    [[[37, 43, 52]], [[41, 46, 52]], [[51, 55, 61]], [[61, 62, 69]], [[63, 63, 78]], [[40, 41, 64]], [[27, 28, 56]], [[28, 28, 53]], [[49, 42, 62]], [[76, 67, 78]], [[76, 70, 77]], [[74, 70, 76]], [[75, 72, 81]], [[73, 71, 80]], [[73, 71, 76]], [[73, 72, 76]], [[74, 72, 76]], [[72, 71, 77]], [[72, 71, 78]], [[73, 71, 78]], [[74, 72, 78]], [[74, 72, 78]], [[78, 75, 82]], [[83, 85, 82]], [[123, 150, 156]], [[75, 95, 137]], [[51, 48, 72]], [[45, 42, 46]], [[43, 40, 44]], [[44, 41, 44]]],
    [[[43, 43, 49]], [[50, 50, 56]], [[53, 52, 58]], [[52, 51, 57]], [[50, 49, 58]], [[41, 43, 58]], [[28, 30, 54]], [[26, 26, 50]], [[43, 35, 56]], [[70, 59, 72]], [[74, 69, 77]], [[74, 70, 76]], [[74, 70, 76]], [[74, 71, 78]], [[73, 71, 78]], [[74, 72, 77]], [[75, 72, 77]], [[75, 72, 77]], [[73, 71, 77]], [[74, 71, 77]], [[74, 72, 78]], [[74, 72, 78]], [[77, 75, 80]], [[85, 85, 75]], [[134, 148, 143]], [[94, 104, 134]], [[50, 48, 66]], [[44, 42, 46]], [[41, 39, 42]], [[43, 39, 42]]],
    [[[30, 29, 34]], [[31, 28, 34]], [[30, 27, 33]], [[28, 27, 32]], [[31, 32, 36]], [[37, 38, 47]], [[31, 31, 52]], [[27, 26, 50]], [[33, 30, 53]], [[53, 48, 63]], [[70, 65, 74]], [[75, 72, 78]], [[73, 72, 77]], [[74, 71, 76]], [[75, 72, 77]], [[75, 71, 77]], [[75, 70, 77]], [[76, 72, 78]], [[74, 71, 76]], [[76, 73, 79]], [[75, 73, 80]], [[75, 73, 80]], [[79, 77, 82]], [[95, 88, 79]], [[193, 168, 169]], [[161, 142, 189]], [[51, 51, 79]], [[43, 42, 47]], [[43, 41, 44]], [[42, 39, 42]]],
    [[[29, 28, 33]], [[30, 28, 33]], [[29, 27, 32]], [[28, 27, 32]], [[28, 28, 30]], [[40, 38, 43]], [[40, 40, 57]], [[25, 28, 50]], [[28, 27, 51]], [[36, 33, 51]], [[63, 55, 67]], [[72, 67, 73]], [[73, 70, 76]], [[72, 72, 77]], [[72, 71, 75]], [[75, 72, 78]], [[75, 72, 81]], [[76, 72, 79]], [[74, 71, 77]], [[75, 73, 78]], [[76, 74, 81]], [[74, 72, 81]], [[78, 73, 84]], [[97, 89, 86]], [[201, 196, 190]], [[195, 188, 255]], [[58, 58, 101]], [[44, 42, 48]], [[43, 40, 44]], [[41, 39, 42]]],
    [[[30, 28, 33]], [[30, 28, 32]], [[28, 28, 32]], [[28, 27, 32]], [[29, 27, 31]], [[35, 32, 36]], [[41, 40, 49]], [[33, 34, 51]], [[28, 28, 50]], [[29, 27, 49]], [[40, 34, 51]], [[60, 54, 66]], [[72, 68, 76]], [[71, 70, 76]], [[71, 71, 76]], [[74, 72, 78]], [[75, 72, 78]], [[76, 73, 80]], [[76, 73, 79]], [[74, 72, 78]], [[75, 73, 80]], [[75, 71, 82]], [[69, 65, 80]], [[81, 78, 73]], [[192, 191, 184]], [[209, 209, 255]], [[58, 57, 90]], [[45, 42, 48]], [[43, 39, 42]], [[42, 39, 42]]],
    [[[32, 30, 36]], [[32, 30, 35]], [[31, 31, 35]], [[31, 31, 36]], [[32, 31, 35]], [[33, 32, 36]], [[39, 35, 40]], [[45, 37, 51]], [[32, 30, 53]], [[27, 28, 50]], [[28, 28, 49]], [[31, 29, 49]], [[57, 52, 67]], [[71, 66, 76]], [[74, 70, 78]], [[72, 71, 78]], [[75, 72, 78]], [[77, 73, 80]], [[77, 73, 80]], [[76, 73, 80]], [[75, 72, 84]], [[71, 64, 79]], [[51, 51, 69]], [[48, 55, 60]], [[175, 177, 175]], [[209, 209, 255]], [[53, 54, 96]], [[44, 42, 48]], [[42, 39, 43]], [[43, 40, 43]]],
    [[[31, 30, 36]], [[31, 29, 34]], [[30, 28, 33]], [[30, 28, 34]], [[30, 30, 34]], [[30, 30, 34]], [[30, 30, 34]], [[38, 37, 48]], [[38, 36, 55]], [[32, 31, 51]], [[27, 27, 50]], [[27, 26, 52]], [[31, 29, 50]], [[40, 38, 54]], [[48, 45, 58]], [[60, 56, 67]], [[64, 61, 73]], [[66, 63, 76]], [[64, 61, 73]], [[57, 57, 69]], [[49, 49, 67]], [[38, 36, 57]], [[34, 34, 55]], [[31, 37, 50]], [[136, 159, 167]], [[136, 156, 236]], [[49, 54, 105]], [[43, 42, 50]], [[44, 40, 45]], [[45, 41, 45]]],
    [[[30, 28, 33]], [[30, 28, 32]], [[29, 27, 31]], [[29, 28, 30]], [[30, 30, 32]], [[29, 29, 32]], [[29, 29, 32]], [[30, 30, 31]], [[40, 38, 40]], [[41, 39, 51]], [[36, 33, 53]], [[29, 29, 53]], [[25, 26, 49]], [[26, 25, 48]], [[28, 26, 50]], [[32, 29, 53]], [[33, 31, 56]], [[34, 32, 57]], [[34, 32, 57]], [[31, 31, 57]], [[29, 29, 55]], [[28, 28, 53]], [[31, 28, 55]], [[39, 38, 49]], [[136, 148, 165]], [[141, 155, 226]], [[57, 62, 104]], [[44, 44, 51]], [[45, 41, 46]], [[46, 41, 44]]],
    [[[31, 29, 33]], [[31, 29, 33]], [[30, 29, 33]], [[29, 28, 31]], [[30, 29, 32]], [[30, 29, 33]], [[30, 27, 30]], [[37, 32, 37]], [[42, 41, 47]], [[52, 51, 56]], [[62, 57, 64]], [[51, 51, 62]], [[45, 48, 60]], [[47, 45, 58]], [[45, 44, 59]], [[45, 45, 59]], [[44, 44, 57]], [[45, 43, 58]], [[46, 44, 59]], [[47, 45, 59]], [[48, 46, 60]], [[50, 48, 62]], [[55, 49, 64]], [[68, 56, 55]], [[182, 168, 170]], [[206, 198, 253]], [[72, 75, 110]], [[45, 46, 55]], [[44, 42, 47]], [[45, 41, 44]]],
    [[[31, 30, 35]], [[31, 31, 34]], [[31, 31, 35]], [[34, 32, 36]], [[35, 32, 35]], [[34, 32, 35]], [[44, 35, 38]], [[68, 47, 52]], [[70, 59, 64]], [[74, 69, 68]], [[76, 71, 71]], [[74, 70, 75]], [[74, 70, 76]], [[77, 71, 78]], [[73, 67, 77]], [[75, 70, 77]], [[74, 69, 74]], [[74, 69, 77]], [[74, 70, 77]], [[76, 72, 78]], [[77, 72, 76]], [[80, 73, 77]], [[82, 72, 78]], [[90, 79, 75]], [[188, 184, 180]], [[216, 217, 255]], [[76, 82, 111]], [[47, 48, 58]], [[44, 42, 47]], [[45, 41, 43]]],
    [[[45, 40, 44]], [[45, 40, 41]], [[44, 39, 40]], [[46, 40, 42]], [[48, 41, 42]], [[45, 41, 42]], [[46, 39, 42]], [[87, 72, 70]], [[77, 71, 64]], [[81, 77, 73]], [[101, 93, 94]], [[104, 97, 107]], [[104, 99, 109]], [[107, 98, 110]], [[84, 73, 86]], [[111, 98, 107]], [[110, 99, 106]], [[108, 99, 108]], [[107, 99, 107]], [[107, 100, 107]], [[106, 101, 104]], [[104, 101, 104]], [[103, 102, 106]], [[102, 102, 114]], [[182, 182, 190]], [[221, 225, 255]], [[69, 79, 102]], [[47, 48, 57]], [[45, 42, 46]], [[45, 42, 43]]],
    [[[43, 33, 35]], [[45, 34, 34]], [[47, 37, 36]], [[44, 35, 33]], [[40, 31, 31]], [[39, 32, 31]], [[46, 40, 32]], [[75, 70, 66]], [[57, 56, 58]], [[64, 62, 58]], [[83, 71, 73]], [[82, 75, 90]], [[74, 70, 80]], [[78, 69, 74]], [[79, 65, 67]], [[86, 68, 73]], [[80, 67, 74]], [[76, 64, 70]], [[81, 69, 76]], [[74, 66, 75]], [[66, 64, 74]], [[62, 67, 72]], [[60, 69, 66]], [[70, 80, 75]], [[174, 177, 174]], [[229, 233, 251]], [[73, 84, 101]], [[46, 48, 58]], [[44, 41, 46]], [[45, 43, 47]]],
    [[[56, 44, 51]], [[51, 37, 38]], [[56, 46, 43]], [[55, 48, 44]], [[48, 43, 44]], [[51, 41, 43]], [[55, 45, 38]], [[77, 71, 70]], [[63, 54, 61]], [[69, 59, 60]], [[73, 63, 68]], [[65, 63, 78]], [[56, 58, 66]], [[63, 56, 59]], [[65, 54, 55]], [[72, 59, 63]], [[70, 62, 68]], [[75, 66, 71]], [[80, 66, 72]], [[66, 63, 72]], [[63, 68, 79]], [[56, 66, 75]], [[49, 58, 62]], [[61, 68, 63]], [[171, 173, 168]], [[250, 252, 255]], [[82, 88, 108]], [[49, 52, 62]], [[45, 47, 52]], [[53, 57, 64]]]
];

In [10]:
input_buffer = allocate(shape=(30*30*3,), dtype=np.int16)
image_np = np.array(image_data, dtype=np.uint8).reshape(-1)

# Make sure the buffer size matches
input_buffer[:] = image_np
print("Input buffer:", input_buffer[:10])  # small slice

Input buffer: [236 240 255 230 231 255 119 125 169  71]


In [11]:
output_buffer0 = allocate(shape=(43,), dtype=np.uint32)
output_buffer1 = allocate(shape=(1,), dtype=np.uint32)

In [12]:
hls_ip.write(0x00,0x81)

In [13]:
hls_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0)
}

In [14]:
dma0.sendchannel.transfer(input_buffer)
dma0.sendchannel.wait()
print("Waiting for X sent")

Waiting for X sent


In [ ]:
dma0.recvchannel.transfer(output_buffer0)
dma1.recvchannel.transfer(output_buffer1)
dma0.recvchannel.wait()
dma1.recvchannel.wait()